In [1]:
import csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm

### Time Dummy

#### I. Apartments

In [53]:
df_apart = pd.read_csv("apartment-for-sale.csv")
for idx, price in enumerate(df_apart['price']):
    if price < 90000000:
        df_apart.at[idx, 'price'] = 90000000
    elif price > 480000000:
        df_apart.at[idx, 'price'] = 480000000
        
# Data preprocessing
df_apart['district'] = df_apart['district'].str.lower().str.strip()
# Drop rows with missing values in the 'date' column
df_apart = df_apart.dropna(subset=['date'])
# Convert the 'date' column to a proper datetime format
df_apart['date'] = pd.to_datetime(df_apart['date'])
# Extracting the quarter from the 'date' column and creating a new 'quarter' column
df_apart['quarter'] = df_apart['date'].dt.quarter
desired_districts = ['gasabo', 'kicukiro', 'nyarugenge']
df_apart_2022 = df_apart[(df_apart['district'].isin(desired_districts)) & (df_apart['date'].dt.year == 2022)]
## Converting price and landsize into log
converted_df_apart_2022 = df_apart_2022.copy()
converted_df_apart_2022 = converted_df_apart_2022[(converted_df_apart_2022["price"] != 0) & (converted_df_apart_2022["landsize"] != 0)]
converted_df_apart_2022[["price", "landsize"]] = np.log(df_apart_2022[["price", "landsize"]])
## Creating dummies
quarters_dummies = pd.get_dummies(converted_df_apart_2022['quarter'], prefix='2022Q')
quarters_dummies.columns = [f'2022Q{col.split("_")[1]}' for col in quarters_dummies.columns]
converted_df_apart_2022 = pd.concat([converted_df_apart_2022, quarters_dummies], axis=1)
converted_df_apart_2022

##Needed columns
selected_columns = [
    "quarter", "landsize", "price", "2022Q1", "2022Q2", "2022Q3", "2022Q4"
]

tdhm_df = converted_df_apart_2022[selected_columns]
tdhm_df

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\internals\blocks.py:402: RuntimeWarning: divide by zero encountered in log
  result = func(self.values, **kwargs)


,quarter,landsize,price,2022Q1,2022Q2,2022Q3,2022Q4
15,4,6.551080,18.315320,0,0,0,1
16,4,6.551080,18.315320,0,0,0,1
17,4,7.090077,19.806975,0,0,0,1
19,3,6.621406,18.420681,0,0,1,0
20,3,5.703782,18.315320,0,0,1,0
21,2,5.703782,19.924758,0,1,0,0
22,2,5.703782,19.485391,0,1,0,0
23,2,6.562444,18.826146,0,1,0,0
24,2,5.703782,19.485391,0,1,0,0
25,2,5.703782,19.673444,0,1,0,0


In [54]:
###Computing coefficients

# Independent var
X = tdhm_df[["2022Q1", "2022Q2", "2022Q3", "2022Q4"]]

X = sm.add_constant(X)
# Dependent var
y = tdhm_df["price"]

# Fit the regression model
model = sm.OLS(y, X).fit()

# Get the coefficients
coefficients = model.params
coefficients

const     15.106396
2022Q1     3.876001
2022Q2     4.262649
2022Q3     3.261604
2022Q4     3.706142
dtype: float64

In [85]:
exponential_coefficients = np.exp(coefficients)
exponential_coefficients

const     3.636006e+06
2022Q1    4.823094e+01
2022Q2    7.099782e+01
2022Q3    2.609136e+01
2022Q4    4.069651e+01
dtype: float64

In [56]:
sum_prices_apart = tdhm_df['price'].sum()
print("-"*49, "\n Sum of prices of apartments in Kigali City - 2022:", sum_prices_apart, 
      "\n" + "-"*49, "\n")

------------------------------------------------- 
 Sum of prices of apartments in Kigali City - 2022: 962.0741208894059 
------------------------------------------------- 



#### II. Houses

In [72]:
## Preprocessing
df_house = pd.read_csv("house_for_sale.csv")

for idx, price in enumerate(df_house['price']):
    if price < 10000000:
        df_house.at[idx, 'price'] = 10000000
    elif price > 250000000:
        df_house.at[idx, 'price'] = 250000000
df_house['district'] = df_house['district'].str.lower().str.strip()
# Drop rows with missing values in the 'date' column
df_house = df_house.dropna(subset=['date'])
# Convert the 'date' column to a proper datetime format
df_house['date'] = pd.to_datetime(df_house['date'])
# Extracting the quarter from the 'date' column and creating a new 'quarter' column
df_house['quarter'] = df_house['date'].dt.quarter
desired_districts = ['gasabo', 'kicukiro', 'nyarugenge']
df_house_2022 = df_house[(df_house['district'].isin(desired_districts)) & (df_house['date'].dt.year == 2022)]

## Converting price and landsize into log
converted_df_house_2022 = df_house_2022.copy()
converted_df_house_2022 = converted_df_house_2022[(converted_df_house_2022["price"] != 0) & (converted_df_house_2022["landsize"] != 0)]
converted_df_house_2022[["price", "landsize"]] = np.log(converted_df_house_2022[["price", "landsize"]])
converted_df_house_2022
## Creating dummies

house_quarters_dummies = pd.get_dummies(converted_df_house_2022['quarter'], prefix='2022Q')
house_quarters_dummies.columns = [f'2022Q{col.split("_")[1]}' for col in house_quarters_dummies.columns]
converted_df_house_2022 = pd.concat([converted_df_house_2022, house_quarters_dummies], axis=1)
converted_df_house_2022
##Needed columns
house_selected_columns = [
    "quarter", "landsize", "price", "2022Q1", "2022Q2", "2022Q3", "2022Q4"
]

house_tdhm_df = converted_df_house_2022[house_selected_columns]
house_tdhm_df

,quarter,landsize,price,2022Q1,2022Q2,2022Q3,2022Q4
5001,4,5.991465,18.132999,0,0,0,1
5002,4,6.214608,18.197537,0,0,0,1
5003,4,5.703782,17.553180,0,0,0,1
5004,4,5.991465,17.766754,0,0,0,1
5005,4,5.991465,18.132999,0,0,0,1
...,...,...,...,...,...,...,...
11756,1,7.090077,18.420681,1,0,0,0
11757,1,6.551080,17.370859,1,0,0,0
11758,1,6.214608,18.197537,1,0,0,0
11759,1,6.214608,18.720785,1,0,0,0


In [73]:
###Computing coefficients

# Independent var
X_h = house_tdhm_df[["2022Q1", "2022Q2", "2022Q3", "2022Q4"]]

X_h = sm.add_constant(X_h)
# Dependent var
y_h = house_tdhm_df["price"]

# Fit the regression model
model_house = sm.OLS(y_h, X_h).fit()

# Get the coefficients
house_coefficients = model_house.params
house_coefficients

const     14.525912
2022Q1     3.577589
2022Q2     3.708030
2022Q3     3.608693
2022Q4     3.631600
dtype: float64

In [74]:
house_exponential_coefficients = np.exp(house_coefficients)
house_exponential_coefficients

const     2.034807e+06
2022Q1    3.578716e+01
2022Q2    4.077342e+01
2022Q3    3.691776e+01
2022Q4    3.777319e+01
dtype: float64

In [75]:
sum_prices_house = house_tdhm_df['price'].sum()
print("-"*49, "\n Sum of prices of houses in Kigali City - 2022:", sum_prices_house, 
      "\n" + "-"*49, "\n")

------------------------------------------------- 
 Sum of prices of houses in Kigali City - 2022: 120863.79153565843 
------------------------------------------------- 



In [76]:
## Computing weights
Total_property_prices = sum_prices_house + sum_prices_apart
print("-"*49, "\n Total Propetry price - 2022:", Total_property_prices, 
      "\n" + "-"*49, "\n")
weight_house = sum_prices_house / Total_property_prices
print("-"*49, "\n House Weight - 2022:", weight_house, 
      "\n" + "-"*49, "\n")
weight_apart = sum_prices_apart / Total_property_prices
print("-"*49, "\n Apartment Weight - 2022:", weight_apart, 
      "\n" + "-"*49, "\n")

------------------------------------------------- 
 Total Propetry price - 2022: 121825.86565654783 
------------------------------------------------- 

------------------------------------------------- 
 House Weight - 2022: 0.9921028747408889 
------------------------------------------------- 

------------------------------------------------- 
 Apartment Weight - 2022: 0.007897125259111154 
------------------------------------------------- 



In [77]:
quarters = ['2022Q1', '2022Q2', '2022Q3', '2022Q4']
# Calculate property aggregate index values for each quarter and store them in a list
property_aggregate_indices = []

for quarter_str in quarters:
#     quarter_int = int(quarter_str[-1])  # Convert '2022Q1' to 1
    if quarter_str in exponential_coefficients and quarter_str in house_exponential_coefficients:
        property_aggregate_index = (exponential_coefficients[quarter_str] * weight_apart) + (house_exponential_coefficients[quarter_str] * weight_house)
        property_aggregate_indices.append(property_aggregate_index)
        continue
    else:
        print(f"No data available for {quarter_str}")

# Combine quarters and aggregated index values
data_aggr = list(zip(quarters, property_aggregate_indices))

# Print aggregated index values for each quarter
print("-" * 49)
print("Aggregated Index Price for each quarter:\n")
for quarter, value in data_aggr:
    print(f"{quarter:<10} | {value:.4f}")
print("-" * 49 + "\n")

-------------------------------------------------
Aggregated Index Price for each quarter:

2022Q1     | 35.8854
2022Q2     | 41.0121
2022Q3     | 36.8323
2022Q4     | 37.7963
-------------------------------------------------



In [111]:
# Specify the CSV file name
csv_file = "tdhm.csv"
quarters =  ['const','2022Q1', '2022Q2', '2022Q3', '2022Q4']

## Apartment
prop_data_apart = pd.DataFrame(list(zip(quarters, exponential_coefficients.values)), columns=['Quarter', 'Value'])
prop_data_apart = prop_data_apart.drop(0)
prop_df_apart = prop_data_apart.reset_index(drop=True)
## House
prop_data_house = pd.DataFrame(list(zip(quarters, house_exponential_coefficients.values)), columns=['Quarter', 'Value'])
prop_data_house = prop_data_house.drop(0)
prop_df_house = prop_data_house.reset_index(drop=True)
prop_df_aggr = pd.DataFrame(data_aggr, columns=['Quarter', 'Aggregated Index'])


# Merge dataframes on 'Quarter' column
prop_df_merged = pd.merge(prop_df_house, prop_df_apart, on='Quarter')
prop_df_merged = pd.merge(prop_df_merged, prop_df_aggr, on='Quarter')
prop_df_merged = prop_df_merged.round(2)

# Open the CSV file in write mode and create a CSV writer
with open(csv_file, mode='w', newline='') as file:
    writer = csv.writer(file)

    # Write the header row
    writer.writerow(["Quarter","Houses", "Apartments","Aggregatedind"])

    # Write the data rows
    for quarter, value in prop_df_merged:
        formatted_value = "{:.2f}".format(value)
        writer.writerow([quarter, formatted_value])

print(f"Data has been saved to {csv_file}")


ValueError: too many values to unpack (expected 2)

In [113]:
import pandas as pd
import csv

# Specify the CSV file name
csv_file = "tdhm.csv"

quarters = ['const', '2022Q1', '2022Q2', '2022Q3', '2022Q4']

## Apartment
prop_data_apart = pd.DataFrame(list(zip(quarters, exponential_coefficients.values)), columns=['Quarter', 'Value'])
prop_data_apart = prop_data_apart.drop(0)
prop_df_apart = prop_data_apart.reset_index(drop=True)

## House
prop_data_house = pd.DataFrame(list(zip(quarters, house_exponential_coefficients.values)), columns=['Quarter', 'Value'])
prop_data_house = prop_data_house.drop(0)
prop_df_house = prop_data_house.reset_index(drop=True)

prop_df_aggr = pd.DataFrame(data_aggr, columns=['Quarter', 'Aggregated Index'])

# Merge dataframes on 'Quarter' column
prop_df_merged = pd.merge(prop_df_house, prop_df_apart, on='Quarter')
prop_df_merged = pd.merge(prop_df_merged, prop_df_aggr, on='Quarter')
prop_df_merged = prop_df_merged.round(2)

# Open the CSV file in write mode and create a CSV writer
with open(csv_file, mode='w', newline='') as file:
    writer = csv.writer(file)

    # Write the header row
    writer.writerow(["Quarter", "Houses", "Apartments", "Aggregated Index"])

    # Write the data rows using iterrows()
    for index, row in prop_df_merged.iterrows():
        quarter = row['Quarter']
        houses_value = "{:.2f}".format(row['Value_x'])
        apartments_value = "{:.2f}".format(row['Value_y'])
        aggregated_index_value = "{:.2f}".format(row['Aggregated Index'])
        
        writer.writerow([quarter, houses_value, apartments_value, aggregated_index_value])

print(f"Data has been saved to {csv_file}")


Data has been saved to tdhm.csv


In [ ]:
import csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm


df_apart = pd.read_csv("apartment-for-sale.csv")
for idx, price in enumerate(df_apart['price']):
    if price < 90000000:
        df_apart.at[idx, 'price'] = 90000000
    elif price > 480000000:
        df_apart.at[idx, 'price'] = 480000000
        
# Data preprocessing
df_apart['district'] = df_apart['district'].str.lower().str.strip()
# Drop rows with missing values in the 'date' column
df_apart = df_apart.dropna(subset=['date'])
# Convert the 'date' column to a proper datetime format
df_apart['date'] = pd.to_datetime(df_apart['date'])
# Extracting the quarter from the 'date' column and creating a new 'quarter' column
df_apart['quarter'] = df_apart['date'].dt.quarter
desired_districts = ['gasabo', 'kicukiro', 'nyarugenge']
df_apart_2022 = df_apart[(df_apart['district'].isin(desired_districts)) & (df_apart['date'].dt.year == 2022)]
## Converting price and landsize into log
converted_df_apart_2022 = df_apart_2022.copy()
converted_df_apart_2022 = converted_df_apart_2022[(converted_df_apart_2022["price"] != 0) & (converted_df_apart_2022["landsize"] != 0)]
converted_df_apart_2022[["price", "landsize"]] = np.log(df_apart_2022[["price", "landsize"]])
## Creating dummies
quarters_dummies = pd.get_dummies(converted_df_apart_2022['quarter'], prefix='2022Q')
quarters_dummies.columns = [f'2022Q{col.split("_")[1]}' for col in quarters_dummies.columns]
converted_df_apart_2022 = pd.concat([converted_df_apart_2022, quarters_dummies], axis=1)
converted_df_apart_2022

##Needed columns
selected_columns = [
    "quarter", "landsize", "price", "2022Q1", "2022Q2", "2022Q3", "2022Q4"
]

tdhm_df = converted_df_apart_2022[selected_columns]
tdhm_df

###Computing coefficients

# Independent var
X = tdhm_df[["2022Q1", "2022Q2", "2022Q3", "2022Q4"]]

X = sm.add_constant(X)
# Dependent var
y = tdhm_df["price"]

# Fit the regression model
model = sm.OLS(y, X).fit()

# Get the coefficients
coefficients = model.params
coefficients
exponential_coefficients = np.exp(coefficients)
exponential_coefficients
sum_prices_apart = tdhm_df['price'].sum()
print("-"*49, "\n Sum of prices of apartments in Kigali City - 2022:", sum_prices_apart, 
      "\n" + "-"*49, "\n")
## Preprocessing
df_house = pd.read_csv("house_for_sale.csv")

for idx, price in enumerate(df_house['price']):
    if price < 10000000:
        df_house.at[idx, 'price'] = 10000000
    elif price > 250000000:
        df_house.at[idx, 'price'] = 250000000
df_house['district'] = df_house['district'].str.lower().str.strip()
# Drop rows with missing values in the 'date' column
df_house = df_house.dropna(subset=['date'])
# Convert the 'date' column to a proper datetime format
df_house['date'] = pd.to_datetime(df_house['date'])
# Extracting the quarter from the 'date' column and creating a new 'quarter' column
df_house['quarter'] = df_house['date'].dt.quarter
desired_districts = ['gasabo', 'kicukiro', 'nyarugenge']
df_house_2022 = df_house[(df_house['district'].isin(desired_districts)) & (df_house['date'].dt.year == 2022)]

## Converting price and landsize into log
converted_df_house_2022 = df_house_2022.copy()
converted_df_house_2022 = converted_df_house_2022[(converted_df_house_2022["price"] != 0) & (converted_df_house_2022["landsize"] != 0)]
converted_df_house_2022[["price", "landsize"]] = np.log(converted_df_house_2022[["price", "landsize"]])
converted_df_house_2022
## Creating dummies

house_quarters_dummies = pd.get_dummies(converted_df_house_2022['quarter'], prefix='2022Q')
house_quarters_dummies.columns = [f'2022Q{col.split("_")[1]}' for col in house_quarters_dummies.columns]
converted_df_house_2022 = pd.concat([converted_df_house_2022, house_quarters_dummies], axis=1)
converted_df_house_2022
##Needed columns
house_selected_columns = [
    "quarter", "landsize", "price", "2022Q1", "2022Q2", "2022Q3", "2022Q4"
]

house_tdhm_df = converted_df_house_2022[house_selected_columns]
house_tdhm_df
###Computing coefficients

# Independent var
X_h = house_tdhm_df[["2022Q1", "2022Q2", "2022Q3", "2022Q4"]]

X_h = sm.add_constant(X_h)
# Dependent var
y_h = house_tdhm_df["price"]

# Fit the regression model
model_house = sm.OLS(y_h, X_h).fit()

# Get the coefficients
house_coefficients = model_house.params
house_coefficients

house_exponential_coefficients = np.exp(house_coefficients)
house_exponential_coefficients

sum_prices_house = house_tdhm_df['price'].sum()
print("-"*49, "\n Sum of prices of houses in Kigali City - 2022:", sum_prices_house, 
      "\n" + "-"*49, "\n")

## Computing weights
Total_property_prices = sum_prices_house + sum_prices_apart
print("-"*49, "\n Total Propetry price - 2022:", Total_property_prices, 
      "\n" + "-"*49, "\n")
weight_house = sum_prices_house / Total_property_prices
print("-"*49, "\n House Weight - 2022:", weight_house, 
      "\n" + "-"*49, "\n")
weight_apart = sum_prices_apart / Total_property_prices
print("-"*49, "\n Apartment Weight - 2022:", weight_apart, 
      "\n" + "-"*49, "\n")

quarters = ['2022Q1', '2022Q2', '2022Q3', '2022Q4']
# Calculate property aggregate index values for each quarter and store them in a list
property_aggregate_indices = []

for quarter_str in quarters:
#     quarter_int = int(quarter_str[-1])  # Convert '2022Q1' to 1
    if quarter_str in exponential_coefficients and quarter_str in house_exponential_coefficients:
        property_aggregate_index = (exponential_coefficients[quarter_str] * weight_apart) + (house_exponential_coefficients[quarter_str] * weight_house)
        property_aggregate_indices.append(property_aggregate_index)
        continue
    else:
        print(f"No data available for {quarter_str}")

# Combine quarters and aggregated index values
data_aggr = list(zip(quarters, property_aggregate_indices))

# Print aggregated index values for each quarter
print("-" * 49)
print("Aggregated Index Price for each quarter:\n")
for quarter, value in data_aggr:
    print(f"{quarter:<10} | {value:.4f}")
print("-" * 49 + "\n")

import pandas as pd
import csv

# Specify the CSV file name
csv_file = "tdhm.csv"

quarters = ['const', '2022Q1', '2022Q2', '2022Q3', '2022Q4']

## Apartment
prop_data_apart = pd.DataFrame(list(zip(quarters, exponential_coefficients.values)), columns=['Quarter', 'Value'])
prop_data_apart = prop_data_apart.drop(0)
prop_df_apart = prop_data_apart.reset_index(drop=True)

## House
prop_data_house = pd.DataFrame(list(zip(quarters, house_exponential_coefficients.values)), columns=['Quarter', 'Value'])
prop_data_house = prop_data_house.drop(0)
prop_df_house = prop_data_house.reset_index(drop=True)

prop_df_aggr = pd.DataFrame(data_aggr, columns=['Quarter', 'Aggregated Index'])

# Merge dataframes on 'Quarter' column
prop_df_merged = pd.merge(prop_df_house, prop_df_apart, on='Quarter')
prop_df_merged = pd.merge(prop_df_merged, prop_df_aggr, on='Quarter')
prop_df_merged = prop_df_merged.round(2)

# Open the CSV file in write mode and create a CSV writer
with open(csv_file, mode='w', newline='') as file:
    writer = csv.writer(file)

    # Write the header row
    writer.writerow(["Quarter", "Houses", "Apartments", "Aggregated Index"])

    # Write the data rows using iterrows()
    for index, row in prop_df_merged.iterrows():
        quarter = row['Quarter']
        houses_value = "{:.2f}".format(row['Value_x'])
        apartments_value = "{:.2f}".format(row['Value_y'])
        aggregated_index_value = "{:.2f}".format(row['Aggregated Index'])
        
        writer.writerow([quarter, houses_value, apartments_value, aggregated_index_value])

print(f"Data has been saved to {csv_file}")
